In [45]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

st.set_page_config(page_title="Predicción de Gastos", page_icon="💸")

df = pd.read_csv("A01615309_Actividad2_Registro-1.csv")

df.columns = df.columns.str.strip()

if "Fecha (dd/mm/aa)" in df.columns:
    df["Fecha (dd/mm/aa)"] = pd.to_datetime(df["Fecha (dd/mm/aa)"], errors="coerce")
else:
    st.error("ERROR: No existe la columna 'Fecha (dd/mm/aa)' en el CSV.")
    st.stop()

df["Año"] = df["Fecha (dd/mm/aa)"].dt.year
df["Mes"] = df["Fecha (dd/mm/aa)"].dt.month
df["Dia"] = df["Fecha (dd/mm/aa)"].dt.day
df["Dia_semana"] = df["Fecha (dd/mm/aa)"].dt.dayofweek

df["Tiempo invertido"] = (
    df["Tiempo invertido"]
    .astype(str)
    .str.replace(" min", "", regex=False)
    .str.strip()
)

df["Tiempo invertido"] = pd.to_numeric(df["Tiempo invertido"], errors="coerce")

df["Costo"] = (
    df["Costo"]
    .astype(str)
    .str.replace("$", "", regex=False)
    .str.strip()
)

df["Costo"] = pd.to_numeric(df["Costo"], errors="coerce")

for col in ["Presupuesto", "No. de personas"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.dropna()

df = df.drop(["Número", "Fecha (dd/mm/aa)"], axis=1)

cat_cols = ["Nombre actividad", "Tipo", "Momento"]
cat_cols = [c for c in cat_cols if c in df.columns]

df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

X = df.drop("Costo", axis=1)
y = df["Costo"]

if y.isnull().sum() > 0:
    st.error("Error: Hay valores no numéricos en 'Costo'.")
    st.stop()

if X.isnull().sum().sum() > 0:
    st.error("Error: X contiene valores NaN.")
    st.stop()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

modelo = DecisionTreeRegressor(max_depth=5, random_state=42)
modelo.fit(X_train, y_train)

st.title("📊 Predicción de Costos de Actividades")
st.write("Modelo robusto y sin errores basado en tus datos personales.")

presupuesto = st.number_input("Presupuesto asignado", min_value=0.0)
tiempo = st.number_input("Tiempo invertido (min)", min_value=0.0)
personas = st.number_input("Número de personas", min_value=1.0)

actividad = st.selectbox("Actividad", X.filter(like="Nombre").columns)
tipo = st.selectbox("Tipo", X.filter(like="Tipo").columns)
momento = st.selectbox("Momento", X.filter(like="Momento").columns)

entrada = np.zeros(len(X.columns))

entrada[X.columns.get_loc("Presupuesto")] = presupuesto
entrada[X.columns.get_loc("Tiempo invertido")] = tiempo
entrada[X.columns.get_loc("No. de personas")] = personas

entrada[X.columns.get_loc(actividad)] = 1
entrada[X.columns.get_loc(tipo)] = 1
entrada[X.columns.get_loc(momento)] = 1

if st.button("Predecir costo"):
    pred = modelo.predict([entrada])[0]
    st.success(f"💵 Costo estimado: **${pred:.2f}**")

st.header("⚖ Equilibrio entre ingresos y gastos")

ingreso = st.number_input("¿Cuánto ganas al mes?", min_value=0.0)
reinversion = st.number_input("Reinversiones o ingresos extra", min_value=0.0)

total = ingreso + reinversion
costo_mensual = df["Costo"].mean() * 30

if st.button("Evaluar equilibrio"):
    if total >= costo_mensual:
        st.success(
            f"✔ Estás en equilibrio (superávit de ${total - costo_mensual:.2f})"
        )
    else:
        deficit = costo_mensual - total
        st.error(f"❌ Déficit de **${deficit:.2f}**")

        reduccion = deficit / costo_mensual * 100
        st.warning(f"Necesitas reducir gastos un **{reduccion:.1f}%**.")


ModuleNotFoundError: No module named 'streamlit'